# **Libs**

In [29]:
# Utils
from src.utils import *

In [30]:
import re
import nltk
import torch
from torch.utils.data import Dataset, DataLoader
from nltk.corpus import stopwords
from collections import Counter

# Загрузка стоп-слов для русского языка
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sefixnep\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Code**

## Зафиксируем seed

In [31]:
set_all_seeds()

In [32]:
df = pd.read_csv("../data/train.csv")

In [33]:
texts = df['comment_text'].tolist()
labels = df['toxic'].tolist()

In [52]:
def preprocess_text(text):
    # Приведение текста к нижнему регистру
    text = text.lower()
    # Удаление пунктуации и чисел
    # text = re.sub(r'[^а-яё\s]', '', text)
    # Токенизация текста
    tokens = text.split()
    # Удаление стоп-слов
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

# Применение функции предобработки к каждому тексту
processed_texts = [preprocess_text(text) for text in texts]

In [ ]:
import pymorphy3
morph = pymorphy3.MorphAnalyzer()

def lemmatize(tokens):
    lemmas = [morph.parse(token)[0].normal_form for token in tokens]
    return lemmas

# Применение лемматизации к токенам
processed_texts = [lemmatize(tokens) for tokens in processed_texts]

In [ ]:
processed_texts

In [53]:
# Создание списка всех токенов
all_tokens = [token for tokens in processed_texts for token in tokens]

# Подсчет частоты каждого слова
word_counts = Counter(all_tokens)

# Создание словаря: слово -> индекс
vocab = {word: idx + 1 for idx, (word, _) in enumerate(word_counts.most_common())}

# Преобразование текстов в последовательности индексов
encoded_texts = [[vocab[word] for word in tokens] for tokens in processed_texts]


In [61]:
from torch.nn.utils.rnn import pad_sequence

# Преобразование списков в тензоры
tensor_texts = [torch.tensor(seq) for seq in encoded_texts]

# Применение паддинга
padded_texts = pad_sequence(tensor_texts, batch_first=True)

In [63]:
labels_tensor = torch.tensor(labels)

In [64]:
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# Создание экземпляра Dataset
dataset = TextDataset(padded_texts, labels_tensor)

# Создание DataLoader для итерации по данным
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [43]:
import torch.nn as nn

class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes):
        super(TextClassificationModel, self).__init__()
        # Слой эмбеддингов
        self.embedding = nn.Embedding(vocab_size + 1, embed_dim, padding_idx=0)
        # Рекуррентный слой (например, LSTM)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        # Полносвязный слой для классификации
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        # Применение слоя эмбеддингов
        x = self.embedding(x)
        # Применение LSTM
        _, (hidden, _) = self.lstm(x)
        # Извлечение последнего скрытого состояния
        hidden = hidden.squeeze(0)
        # Применение полносвязного слоя
        out = self.fc(hidden)
        return out

# Параметры модели
vocab_size = len(vocab)
embed_dim = 128
hidden_dim = 256
num_classes = len(set(labels))

# Инициализация модели
model = TextClassificationModel(vocab_size, embed_dim, hidden_dim, num_classes)